In [1]:
import numpy as np
from scipy.stats import logser, norm

In [2]:
t_0 = 0.07
t_1 = 0.09

np.random.seed(42)
sample = logser.rvs(t_0, size=1000)

## Вычисление функции отношения правдоподобия

In [3]:
def likelihood_ratio(t_0, t_1, sample):
    n = len(sample)
    return (np.log(1 - t_0) / np.log(1 - t_1)) ** n * ((t_1 / t_0) ** np.sum(sample))

In [4]:
print(f'theta_0 = {t_0}; theta_1 = {t_1} -> likelihood_ratio = {likelihood_ratio(t_0, t_1, sample)}')

theta_0 = 0.07; theta_1 = 0.09 -> likelihood_ratio = 0.18796018525782482


In [5]:
print(f'Проверка: {likelihood_ratio(t_1, t_0, sample) * likelihood_ratio(t_0, t_1, sample)}')

Проверка: 1.0000000000001095


## Применение критерия

In [6]:
def get_mean(t):
    return -t / (np.log(1 - t)* (1 - t))

def get_var(t):
    return -t * (np.log(1 - t) + t) / ((1 - t)**2 * (np.log(1 - t))**2)

def get_g_alpha(alpha):
    return - norm.ppf(alpha)

In [7]:
def check_crit(sample, t_0, alpha):
    n = len(sample)
    left_part = (np.sum(sample) - n * get_mean(t_0)) / (np.sqrt(n) * get_var(t_0))
    if(left_part >= get_g_alpha(alpha)):
        print('Принимаем гипотезу H_1')
    else:
        print('Принимаем гипотезу H_0')

In [22]:
check_crit(sample, t_0, alpha=0.01)

Принимаем гипотезу H_0


In [20]:
check_crit(logser.rvs(t_1, size=1000), t_0, alpha=0.01)

Принимаем гипотезу H_1


In [9]:
def get_beta(n, t_0, t_1, alpha):
    return norm.cdf(np.sqrt(n) * (get_mean(t_0) - get_mean(t_1)) / get_var(t_1) + get_g_alpha(alpha) * get_var(t_0) / get_var(t_1))

In [10]:
get_beta(len(sample), t_0, t_1, alpha=0.01)

1.2623251776628952e-07

In [11]:
get_beta(len(sample), t_0, t_1, alpha=0.05)

7.265906717272991e-09

In [12]:
get_beta(len(sample), t_0, t_1, alpha=0.1)

1.4295743860678404e-09

## Вычисление минимального необходимого количества материала при фиксации минимального возможного значения ошибок первого и второго рода

In [13]:
def get_count_material(t_0, t_1, alpha, beta):
    n = (get_var(t_1) * norm.ppf(beta) + get_var(t_0) * norm.ppf(alpha)) ** 2 / (get_mean(t_0) - get_mean(t_1))**2
    return np.ceil(n)

In [14]:
get_count_material(t_0, t_1, alpha=0.1, beta=0.1)

106.0

In [15]:
get_count_material(t_0, t_1, alpha=0.05, beta=0.05)

175.0

In [16]:
get_count_material(t_0, t_1, alpha=0.01, beta=0.01)

349.0

In [17]:
get_count_material(t_0, t_1, alpha=0.001, beta=0.001)

615.0

In [18]:
alpha = 0.0001
beta = 0.0001

n = get_count_material(t_0, t_1, alpha, beta)
print(f'n* = {n} | beta для этого n: {get_beta(n, t_0, t_1, alpha)} | Проверка на beta: {get_beta(n, t_0, t_1, alpha) <= beta}')
print(f'n - действительно минимальное число -> все значения < n* дают значение beta > фиксированного: {all(get_beta(i, t_0, t_1, alpha) > beta for i in range(1, int(n)))}')

n* = 890.0 | beta для этого n: 9.93221043361567e-05 | Проверка на beta: True
n - действительно минимальное число -> все значения < n* дают значение beta > фиксированного: True
